# All

In [1]:
from google.colab import auth
auth.authenticate_user()

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
%%capture
!pip install datasets

In [2]:
# %%capture

!pip install --force-reinstall git+https://github.com/CaitlinJCorbin/transformers

  Cloning https://github.com/CaitlinJCorbin/transformers to /tmp/pip-req-build-bcbk9hg4
  Running command git clone -q https://github.com/CaitlinJCorbin/transformers /tmp/pip-req-build-bcbk9hg4
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
    Preparing wheel metadata ... done
  Using cached PyYAML-5.4.1-cp37-cp37m-manylinux1_x86_64.whl (636 kB)
  Using cached regex-2021.8.3-cp37-cp37m-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (722 kB)
  Using cached tokenizers-0.10.3-cp37-cp37m-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_12_x86_64.manylinux2010_x86_64.whl (3.3 MB)
  Using cached requests-2.26.0-py2.py3-none-any.whl (62 kB)
  Using cached huggingface_hub-0.0.15-py3-none-any.whl (43 kB)
  Using cached importlib_metadata-4.6.3-py3-none-any.whl (17 kB)
  Using cached filelock-3.0.12-py3-none-any.whl (7.6 kB)
  Using cached numpy-1.21.1-cp37-cp37m-manylinux_2_12_x86_64.manylinux2010_x86_64.whl (15.7 MB)
  Using cached packaging-21.0-py3

In [3]:
from transformers import EncoderDecoderModel, DistilBertTokenizer, DistilBertConfig, DistilBertModel
import torch
import numpy as np

In [4]:
if torch.cuda.is_available():
  device = torch.device('cuda')
  print(torch.cuda.get_device_name())
else:
  device = torch.device('cpu')
  

Tesla T4


## Data #1


In [6]:
!gsutil -m cp /content/drive/MyDrive/immunolinguistics/frames_converted/*otsu.*.txt.gz . 

CommandException: No URLs matched: /content/drive/MyDrive/immunolinguistics/frames_converted/*otsu.*.txt.gz
CommandException: 1 file/object could not be transferred.


In [7]:
!gzip -d *.gz

gzip: *.gz: No such file or directory


In [8]:
!mkdir otsu
!gsutil -m cp *txt otsu/ 

mkdir: cannot create directory ‘otsu’: File exists
CommandException: No URLs matched: *txt
CommandException: 1 file/object could not be transferred.


In [10]:
import os 
!mkdir otsu_cleaned
for file_name in os.listdir('/content/otsu'):
  with open(f'otsu/{file_name}') as file:
    with open(f'otsu_cleaned/{file_name}', 'w') as out_file:
      for line in file:
        line = ' '.join(line.strip().split()[1:-1])
        out_file.write(f'{line}\n')

mkdir: cannot create directory ‘otsu_cleaned’: File exists


IsADirectoryError: ignored

In [ ]:
!gsutil -m cp otsu_cleaned/* gs://cytereader/otsu/ 

## Train

In [11]:
!pip install tokenizers
#!mkdir cellAttention

In [12]:
#from google.colab import files
#uploaded = files.upload()

In [13]:
!gsutil cp gs://cytereader/preprocessed_cell_corpus_0.txt . 
#!gsutil cp gs://cytereader/preprocessed_cell_corpus_1.txt . 

CommandException: No URLs matched: gs://cytereader/preprocessed_cell_corpus_0.txt


In [ ]:
#from tokenizers import BertWordPieceTokenizer

# wb_tokenizer = BertWordPieceTokenizer(clean_text=True,
#                                       strip_accents=True, lowercase=True)

# wb_tokenizer.train(['preprocessed_cell_corpus_0.txt', 'preprocessed_cell_corpus_1.txt'],
#                    vocab_size=10000, min_frequency=2,
#                    special_tokens=["[PAD]", "[UNK]", "[CLS]", "[SEP]", "[MASK]"])
# wb_tokenizer.save_model("./cellAttention")

In [14]:
from transformers import BertConfig, AutoTokenizer, BertTokenizer, BertLMHeadModel, DistilBertForMaskedLM
from transformers.models.bert.tokenization_bert import CellBertTokenizer

configuration = DistilBertConfig(vocab_size=1000)
model = DistilBertForMaskedLM(configuration)

#tokenizer = BertTokenizer.from_pretrained("./cellAttention", max_len=64)
#tokenizer = CellBertTokenizer.from_pretrained('./cellAttention/',vocab_file="vocab.txt")


In [15]:
!mkdir cellAttention
!gsutil cp gs://cytereader/vocab.txt cellAttention/
tokenizer = CellBertTokenizer.from_pretrained('./cellAttention',vocab_file="./cellAttention/vocab.txt")

#tokenizer = CellBertTokenizer.from_pretrained('./cellAttention/',vocab_file="vocab.txt")
#model.num_parameters()

mkdir: cannot create directory ‘cellAttention’: File exists
Copying gs://cytereader/vocab.txt...
/ [1 files][  978.0 B/  978.0 B]                                                
Operation completed over 1 objects/978.0 B.                                      


file ./cellAttention/config.json not found


In [16]:
tokenizer.vocab
tokenizer.encode('CD45+ CD196_CCR6+'.lower())

[2, 97, 1, 3]

In [ ]:
#cnt = 0
#with open('small.txt','w') as file:
 # for line in open("preprocessed_cell_corpus_0.txt"):
  #  file.write(line.lower())
    # cnt += 1
    # if cnt == 128:
    #   break

In [17]:
!mkdir otsu
!gsutil -m cp -r gs://cytereader/otsu/ otsu/

mkdir: cannot create directory ‘otsu’: File exists
Copying gs://cytereader/otsu/Z23S-otsu.0.txt...
Copying gs://cytereader/otsu/Z23S-otsu.13.txt...
Copying gs://cytereader/otsu/Z23S-otsu.10.txt...
Copying gs://cytereader/otsu/Z23S-otsu.14.txt...
Copying gs://cytereader/otsu/Z23S-otsu.15.txt...
Copying gs://cytereader/otsu/Z23S-otsu.11.txt...
Copying gs://cytereader/otsu/Z23S-otsu.1.txt...
Copying gs://cytereader/otsu/Z23S-otsu.12.txt...
Copying gs://cytereader/otsu/Z23S-otsu.16.txt...
Copying gs://cytereader/otsu/Z23S-otsu.2.txt...
Copying gs://cytereader/otsu/Z23S-otsu.3.txt...
Copying gs://cytereader/otsu/Z23S-otsu.4.txt...
Copying gs://cytereader/otsu/Z23S-otsu.5.txt...
Copying gs://cytereader/otsu/Z23S-otsu.6.txt...
Copying gs://cytereader/otsu/Z23S-otsu.7.txt...
Copying gs://cytereader/otsu/Z23S-otsu.8.txt...
Copying gs://cytereader/otsu/Z23S-otsu.9.txt...
Copying gs://cytereader/otsu/Z2L2-otsu.0.txt...
Copying gs://cytereader/otsu/Z2L2-otsu.1.txt...
Copying gs://cytereader/otsu/Z

In [18]:
%%time
from transformers import LineByLineTextDataset
from datasets import load_dataset
import os 

files_name = ['otsu/otsu/' + x for x in os.listdir('/content/otsu/otsu/')][0:2]
dataset = load_dataset("text", data_files=files_name, split='train')
max_length = 128
batch_size=64 

def tokenize_function(examples):
    tokenizer_ = tokenizer([x.lower() for x in examples['text']], max_length=max_length, truncation=True, padding='max_length')
    return tokenizer_



train_data_batch = dataset.map(
    tokenize_function, 
    batched=True, 
    batch_size=batch_size,
)

Using custom data configuration default-352a27152c25d5f2


0 tables [00:00, ? tables/s]

Dataset text downloaded and prepared to /root/.cache/huggingface/datasets/text/default-352a27152c25d5f2/0.0.0/e16f44aa1b321ece1f87b07977cc5d70be93d69b20486d6dacd62e12cf25c9a5. Subsequent calls will reuse this data.


  0%|          | 0/31250 [00:00<?, ?ba/s]

CPU times: user 2min 27s, sys: 5.07 s, total: 2min 33s
Wall time: 2min 34s


In [19]:
from transformers.data.data_collator import DataCollatorForLanguageModeling, DataCollatorForSOP, DataCollatorForNetutralCellModeling

data_collator = DataCollatorForNetutralCellModeling(
    tokenizer=tokenizer, ncm=True, mlm_probability=0.15)

In [20]:
from transformers import Trainer, TrainingArguments

training_args = TrainingArguments(
    output_dir="./cellAttention",
    overwrite_output_dir=True,
    num_train_epochs=1,
    per_device_train_batch_size=32,
    save_steps=10_000,
    learning_rate=1e-4,
    save_total_limit=2,
    prediction_loss_only=True,
)

trainer = Trainer(
    model=model,
    args=training_args,
    data_collator=data_collator,
    train_dataset=train_data_batch,
)

In [ ]:
# %%time
trainer.train()

The following columns in the training set  don't have a corresponding argument in `DistilBertForMaskedLM.forward` and have been ignored: text, token_type_ids.
***** Running training *****
  Num examples = 2000000
  Num Epochs = 1
  Instantaneous batch size per device = 32
  Total train batch size (w. parallel, distributed & accumulation) = 32
  Gradient Accumulation steps = 1
  Total optimization steps = 62500


Step,Training Loss
500,0.530900
1000,0.387600
1500,0.367300
2000,0.360900
2500,0.356000
3000,0.351900
3500,0.355300
4000,0.345800
4500,0.346800
5000,0.340000


Saving model checkpoint to ./cellAttention/checkpoint-10000
Configuration saved in ./cellAttention/checkpoint-10000/config.json
Model weights saved in ./cellAttention/checkpoint-10000/pytorch_model.bin
Saving model checkpoint to ./cellAttention/checkpoint-20000
Configuration saved in ./cellAttention/checkpoint-20000/config.json
Model weights saved in ./cellAttention/checkpoint-20000/pytorch_model.bin


In [ ]:
trainer.save_model("./cellAttention")

In [ ]:
from transformers import pipeline

fill_mask = pipeline(
    "fill-mask",
    model="./cellAttention",
    tokenizer="./cellAttention"
)

In [ ]:
fill_mask("CD45+ CD196_CCR6+ CD181_CXCR1- HLA_DR- CD15- CD31_PECAM1- CD8a- CD182_CXCR2[MASK] CD66ace- CD63- CD14- CD66b- CD62L_Lselectin- CD3+ CD27- CD86+ CD10- CD197_CCR7+ CD28- CD11c- CD33- CD161- CD45RO- CD24- CD38+ CD278_ICOS- CD32- CD152_CTLA4+ IgM+ CD184_CXCR4+ CD279_PD1- CD56+ CD16-")

In [ ]:
input = 'CD45+ CD196_CCR6+ CD181_CXCR1- HLA_DR- CD15-'
masked_input = 'CD45+ CD196_CCR6+ CD181_CXCR1[MASKED] HLA_DR- CD15-'

output = 'CD45+ CD196_CCR6+ CD181_CXCR1-/+ HLA_DR- CD15-'

In [ ]:
encode = tokenizer.encode('CD45+')
encode

In [ ]:
tokenizer.decode(encode)


In [ ]:
tokenizer.convert_ids_to_tokens([56, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37,
        37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37,
        37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37,
        37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37,
        37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37, 37,
        37, 61])

In [ ]:
tokenizer.convert_tokens_to_ids('CD45+ CD196_CCR6+ CD181_CXCR1- HLA_DR- CD15- CD31_PECAM1- CD8a- CD182_CXCR2- IgA- CD66ace- CD63- CD14- CD66b- CD62L_Lselectin- CD3+ CD27- CD86+ CD10- CD197_CCR7+ CD28- CD11c- CD33- CD161- CD45RO- CD24- CD38+ CD278_ICOS- CD32- CD152_CTLA4+ IgM+ CD184_CXCR4+ CD279_PD1- CD56+ CD16-'.split())

In [ ]:
tokenizer.convert_tokens_to_ids(['CD45+', 'CD196_CCR6+', 'CD181_CXCR1-', 'HLA_DR-', 'CD15-', 'CD31_PECAM1-', 'CD8a-', 'CD182_CXCR2-', 'IgA-', 'CD66ace-', 'CD63-', 'CD14-', 'CD66b-', 'CD62L_Lselectin-', 'CD3+', 'CD27-', 'CD86+', 'CD10-', 'CD197_CCR7+', 'CD28-', 'CD11c-', 'CD33-', 'CD161-', 'CD45RO-', 'CD24-', 'CD38+', 'CD278_ICOS-', 'CD32-', 'CD152_CTLA4+', 'IgM+', 'CD184_CXCR4+', 'CD279_PD1-', 'CD56+', 'CD16-']
)

In [ ]:
!gsutil cp -r  /content/cellAttention/*.bin gs://cytereader/longphan/test_bert_0/
!gsutil cp -r  /content/cellAttention/config.json gs://cytereader/longphan/test_bert_0/
!gsutil cp -r  /content/cellAttention/vocab.txt gs://cytereader/longphan/test_bert_0/